# <font color="#418FDE" size="6.5" uppercase>**Environment Setup**</font>

>Last update: 20260125.
    
By the end of this Lecture, you will be able to:
- Install TensorFlow 2.20.0 and verify the installation on local or cloud hardware. 
- Configure GPU or TPU acceleration for TensorFlow 2.20.0 where available. 
- Set up a reproducible project structure for TensorFlow experiments used in this course. 


## **1. TensorFlow Installation Essentials**

### **1.1. TensorFlow Install Commands**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_01_01.jpg?v=1769358617" width="250">



>* Use package manager to install TensorFlow 2.20.0
>* Activate an isolated environment and run install command

>* Cloud notebooks still need explicit TensorFlow installs
>* Always pin version 2.20.0 in every environment

>* Install may be slow; watch messages carefully
>* Handle errors, rerun version-pinned command as needed



### **1.2. Handling CUDA and cuDNN versions**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_01_02.jpg?v=1769358635" width="250">



>* TensorFlow GPU needs specific CUDA and cuDNN
>* Version mismatches cause errors, so match requirements

>* Check TensorFlow’s official CUDA, cuDNN compatibility matrix
>* Match local or cloud environments to those versions

>* Treat CUDA and cuDNN as pinned dependencies
>* Use documented, shared setups for consistent, stable runs



### **1.3. Confirm TensorFlow Version**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_01_03.jpg?v=1769358650" width="250">



>* Check TensorFlow version immediately after installation
>* Prevents old versions causing confusing errors

>* Check TensorFlow version inside your active environment
>* Avoid hidden version mismatches and subtle bugs

>* Routinely recheck TensorFlow version when sessions change
>* Consistent versions improve reproducibility, collaboration, and debugging



In [ ]:
#@title Python Code - Confirm TensorFlow Version

# This script confirms your TensorFlow installation version.
# Run it after installing TensorFlow in your environment.
# It helps ensure this course version is available.

# If TensorFlow is missing, uncomment and run this line.
# !pip install tensorflow==2.20.0.

# Import the standard library module for system details.
import sys

# Try importing TensorFlow and handle missing installation.
try:
    import tensorflow as tf
except ImportError as exc:
    print("TensorFlow is not installed in this environment.")
    print("Install tensorflow==2.20.0, then rerun this script.")
    raise SystemExit(1) from exc

# Get the currently active Python version string.
python_version = sys.version.split()[0]

# Read the TensorFlow version string from the package.
tf_version = getattr(tf, "__version__", "unknown")

# Define the expected TensorFlow version for this course.
expected_version = "2.20.0"

# Print concise environment and version information.
print("Python version:", python_version)
print("TensorFlow version:", tf_version)

# Check whether the installed version matches expectations.
if tf_version == expected_version:
    print("TensorFlow version matches the course requirement.")
else:
    print("Warning: TensorFlow version does not match expected.")
    print("Expected:", expected_version, "but found:", tf_version)

# Try to create a tiny constant tensor as a smoke test.
try:
    sample_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
except Exception as exc:
    print("TensorFlow imported but basic operations failed.")
    raise SystemExit(1) from exc

# Validate the tensor shape to ensure basic functionality.
if tuple(sample_tensor.shape) != (2, 2):
    print("Unexpected tensor shape, something is misconfigured.")
    raise SystemExit(1)

# Print a short confirmation that TensorFlow runs simple operations.
print("TensorFlow ran a simple tensor operation successfully.")



## **2. TensorFlow Hardware Acceleration**

### **2.1. Detecting Available Devices**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_02_01.jpg?v=1769358707" width="250">



>* Check which devices TensorFlow can actually use
>* Device detection defines your training speed and limits

>* Ask TensorFlow which CPU and GPUs exist
>* Check early to spot missing or misconfigured devices

>* Use device output to diagnose hardware issues
>* Plan workload distribution and avoid wasted compute time



In [ ]:
#@title Python Code - Detecting Available Devices

# This script shows TensorFlow device detection basics.
# You will learn how to list available devices.
# Use this to confirm CPU GPU or TPU visibility.

# !pip install tensorflow==2.20.0.

# Import standard library module for environment access.
import os

# Try importing TensorFlow and handle missing installation.
try:
    import tensorflow as tf
except ImportError as e:
    print("TensorFlow is not installed in this environment.")
    print("Please install tensorflow==2.20.0 and rerun.")
    raise SystemExit(1)

# Print TensorFlow version to confirm expected release.
print("TensorFlow version:", tf.__version__)

# Define a helper to print a short device summary.
def print_device_list(label, devices):
    print(label, "count:", len(devices))
    for d in devices:
        print(" -", d.name, "|", d.device_type)

# List logical devices that TensorFlow can currently use.
logical_devices = tf.config.list_logical_devices()
print_device_list("Logical devices", logical_devices)

# List physical devices detected on this machine.
physical_devices = tf.config.list_physical_devices()
print_device_list("Physical devices", physical_devices)

# List only GPU devices if any are available.
gpu_devices = tf.config.list_physical_devices("GPU")
print_device_list("GPU devices", gpu_devices)

# List only TPU devices if any are available.
tpu_devices = tf.config.list_physical_devices("TPU")
print_device_list("TPU devices", tpu_devices)

# Show a simple message summarizing acceleration availability.
if gpu_devices:
    print("At least one GPU is available for TensorFlow.")
elif tpu_devices:
    print("A TPU device is available for TensorFlow.")
else:
    print("Only CPU devices are currently visible to TensorFlow.")




### **2.2. Managing GPU Memory**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_02_02.jpg?v=1769358753" width="250">



>* GPU memory is limited and easily exhausted
>* Understand usage to avoid crashes and share fairly

>* TensorFlow can reserve or grow GPU memory
>* Flexible allocation lets multiple users share GPUs

>* Model design choices directly affect GPU memory
>* Adjust settings and monitor usage to stay stable



In [ ]:
#@title Python Code - Managing GPU Memory

# This script demonstrates basic GPU memory ideas.
# It uses only standard Python and prints summaries.
# Adjust concepts later when using real TensorFlow.

# No external libraries are required for this script.
# In Colab you would install TensorFlow separately.
# Example install line is commented out intentionally.
# !pip install tensorflow==2.20.0.

# Import os to inspect environment variables safely.
import os

# Define a helper to format byte values nicely.
def format_bytes(num_bytes):
    units = ["B", "KB", "MB", "GB"]
    size = float(num_bytes)
    unit_index = 0
    while size >= 1024 and unit_index < len(units) - 1:
        size /= 1024.0
        unit_index += 1
    return f"{size:.2f} {units[unit_index]}"

# Simulate total GPU memory for teaching purposes.
SIMULATED_TOTAL_GPU_BYTES = 4 * 1024 * 1024 * 1024

# Simulate TensorFlow reserving almost all GPU memory.
def simulate_greedy_allocation(total_bytes, reserve_ratio):
    reserved = int(total_bytes * reserve_ratio)
    free_bytes = total_bytes - reserved
    return reserved, free_bytes

# Simulate gradual growth of GPU memory usage.
def simulate_growth_steps(total_bytes, step_ratio, steps):
    used_list = []
    used = 0
    for _ in range(steps):
        increment = int(total_bytes * step_ratio)
        used = min(total_bytes, used + increment)
        used_list.append(used)
    return used_list

# Show a header explaining the simple simulation.
print("GPU memory management concept simulation (no real GPU).")

# Demonstrate greedy allocation behavior numerically.
reserved_greedy, free_greedy = simulate_greedy_allocation(
    SIMULATED_TOTAL_GPU_BYTES, 0.9
)
print("If TensorFlow reserves 90% upfront:")
print("  Reserved:", format_bytes(reserved_greedy))
print("  Free for others:", format_bytes(free_greedy))

# Demonstrate gradual growth over several training steps.
steps_used = simulate_growth_steps(
    SIMULATED_TOTAL_GPU_BYTES, 0.15, 4
)
print("\nIf TensorFlow grows usage gradually:")
for index, used_bytes in enumerate(steps_used, start=1):
    print(f"  After step {index}: {format_bytes(used_bytes)} used")

# Show how batch size can affect memory usage conceptually.
example_batch_sizes = [8, 16, 32]

# Assume each sample needs a fixed memory amount.
BYTES_PER_SAMPLE = 64 * 1024 * 1024

# Print estimated memory usage for each batch size.
for batch in example_batch_sizes:
    estimated = batch * BYTES_PER_SAMPLE
    print(
        f"Batch {batch} needs about {format_bytes(estimated)} GPU memory"
    )




### **2.3. Colab and Cloud TPUs**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_02_03.jpg?v=1769358800" width="250">



>* Select GPU or TPU runtimes, then verify
>* Rerun checks whenever Colab sessions restart

>* Cloud TPUs act as remote network accelerators
>* Optimize supported ops and data movement carefully

>* Handle ephemeral sessions with repeatable setup steps
>* Standardize environments so teams get consistent results



In [ ]:
#@title Python Code - Colab and Cloud TPUs

# This script checks Colab GPU or TPU availability.
# It helps verify TensorFlow hardware acceleration quickly.
# Run it after selecting GPU or TPU runtime.

# !pip install tensorflow==2.20.0.

# Import standard libraries safely.
import os
import random
import numpy as np

# Set deterministic seeds for reproducibility.
random.seed(42)
np.random.seed(42)

# Try importing TensorFlow with clear error handling.
try:
    import tensorflow as tf
except ImportError as e:
    raise SystemExit("TensorFlow is not installed correctly.") from e

# Print TensorFlow version in one short line.
print("TensorFlow version:", tf.__version__)

# Check if any GPU devices are visible.
gpus = tf.config.list_physical_devices("GPU")
print("GPUs available:", len(gpus))

# Check if any TPU devices are visible.
tpus = tf.config.list_logical_devices("TPU")
print("TPUs available:", len(tpus))

# Decide which device name to use for testing.
if tpus:
    test_device = "/TPU:0"
elif gpus:
    test_device = "/GPU:0"
else:
    test_device = "/CPU:0"

# Print the chosen device for the small test.
print("Using device for test:", test_device)

# Create a tiny deterministic tensor for computation.
values = tf.constant([[1.0, 2.0], [3.0, 4.0]])

# Validate tensor shape before running operations.
if values.shape != (2, 2):
    raise ValueError("Unexpected tensor shape for test matrix.")

# Run a small matrix multiplication on the chosen device.
with tf.device(test_device):
    result = tf.matmul(values, values)

# Convert result to numpy for safe printing.
result_np = result.numpy()

# Print the result and device confirmation.
print("Test matrix multiplication result:")
print(result_np)

# Final confirmation message about hardware acceleration.
print("If device is GPU or TPU, acceleration works.")



## **3. TensorFlow Project Setup**

### **3.1. Project Folder Structure**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_03_01.jpg?v=1769358886" width="250">



>* Organized folders make TensorFlow projects easy to navigate
>* Good structure improves reproducibility and reduces mental effort

>* Match folders to data, models, and outputs
>* Structure experiments so results trace back to settings

>* Use the same folder layout every project
>* Standard structure improves collaboration, automation, and reproducibility



### **3.2. Virtual Environments Basics**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_03_02.jpg?v=1769358906" width="250">



>* Use virtual environments to isolate project dependencies
>* Run different TensorFlow versions without conflicts or bugs

>* Record and share exact package versions per project
>* Recreate identical environments later for consistent results

>* Create and use a fresh environment per project
>* Prevents conflicts and keeps experiments consistent everywhere



In [ ]:
#@title Python Code - Virtual Environments Basics

# This script explains virtual environments conceptually.
# It does not create or manage real environments.
# Use it to visualize isolated project dependencies.

# Define a simple class representing a virtual environment.
class VirtualEnv:
    # Initialize with a name and empty packages dictionary.
    def __init__(self, name):
        self.name = name
        self.packages = {}

    # Install or update a package with a specific version.
    def install(self, package, version):
        self.packages[package] = version

    # Return a formatted description of this environment.
    def describe(self):
        lines = [f"Environment: {self.name}"]
        for pkg, ver in sorted(self.packages.items()):
            lines.append(f"  {pkg}=={ver}")
        return "\n".join(lines)

# Create two separate virtual environments for demonstration.
course_env = VirtualEnv("course_tf_env")
research_env = VirtualEnv("research_tf_env")

# Install TensorFlow and helpers in the course environment.
course_env.install("tensorflow", "2.20.0")
course_env.install("numpy", "1.26.0")

# Install different versions in the research environment.
research_env.install("tensorflow", "2.22.0")
research_env.install("numpy", "2.0.0")

# Show that each environment keeps its own versions.
print("Course environment packages:")
print(course_env.describe())

# Print a blank line to separate the two outputs.
print()

# Show the independent configuration of the research environment.
print("Research environment packages:")
print(research_env.describe())




### **3.3. Git Workflow for TensorFlow**

<img src="https://cdn.jsdelivr.net/gh/mhrafiei/contents@main/LFF/Master TensorFlow 2.20.0/Module_01/Lecture_B/image_03_03.jpg?v=1769358941" width="250">



>* Git organizes TensorFlow work and tracks changes
>* Commit history enables comparison, debugging, and reproducibility

>* Use branches to separate stable code and experiments
>* Commit small changes, review branches, merge successful work

>* Version control configs, code, and experiment behavior
>* Avoid large artifacts; keep concise experiment logs



# <font color="#418FDE" size="6.5" uppercase>**Environment Setup**</font>


In this lecture, you learned to:
- Install TensorFlow 2.20.0 and verify the installation on local or cloud hardware. 
- Configure GPU or TPU acceleration for TensorFlow 2.20.0 where available. 
- Set up a reproducible project structure for TensorFlow experiments used in this course. 

In the next Lecture (Lecture C), we will go over 'TF 2.20 Overview'